# Tuto to find The sources and Objects in LSSTComCam

- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab/IN2P3/CNRS
- member : DESC, rubin-inkind
- creation date : 2025-05-01
- last update : 2025-05-03

## Note on work done on LSSTComCam Commissioning : https://sitcomtn-149.lsst.io/
## Note Data Product definition Document : https://lse-163.lsst.io/

In [ ]:
import sys
import matplotlib.pyplot as plt
import lsst.afw.display as afwDisplay
import numpy as np
import pandas as pd
from astropy.time import Time
# %matplotlib widget

In [ ]:
import lsst.geom as geom
from lsst.geom import SpherePoint, degrees, Point2D, Point2I
from lsst.skymap import PatchInfo, Index2D

In [ ]:
plt.rcParams["figure.figsize"] = (10, 6)
plt.rcParams["axes.labelsize"] = "x-large"
plt.rcParams["axes.titlesize"] = "x-large"
plt.rcParams["xtick.labelsize"] = "x-large"
plt.rcParams["ytick.labelsize"] = "x-large"

In [ ]:
import gc

In [ ]:
import traceback

In [ ]:
# Define butler
from lsst.daf.butler import Butler

In [ ]:
!eups list lsst_distrib

In [ ]:
def nJy_to_ab_mag(f_njy):
    """Convert scalar or array flux in nJy to AB magnitude."""
    f_njy = np.asarray(f_njy)
    mag = np.full_like(f_njy, fill_value=np.nan, dtype=float)
    mask = f_njy > 0
    mag[mask] = -2.5 * np.log10(f_njy[mask]) + 31.4
    return mag


def nJy_err_to_ab_err(f_njy, f_err):
    """Propagate flux error to magnitude error."""
    f_njy = np.asarray(f_njy)
    f_err = np.asarray(f_err)
    mag_err = np.full_like(f_njy, fill_value=np.nan, dtype=float)
    mask = (f_njy > 0) & (f_err > 0)
    mag_err[mask] = (2.5 / np.log(10)) * (f_err[mask] / f_njy[mask])
    return mag_err

In [ ]:
def ab_mag_to_nJy(mag_ab):
    """Convert AB magnitude to flux in nanojanskys."""
    return 10 ** ((31.4 - mag_ab) / 2.5)

## RubinTV, Campaigns , quicklook
- RubinTV : https://usdf-rsp.slac.stanford.edu/rubintv/summit-usdf/lsstcam
- https://rubinobs.atlassian.net/wiki/spaces/LSSTCOM/pages/467370016/LSSTCam+Commissioning+Planning
- LSSTCam DM campaign : https://rubinobs.atlassian.net/wiki/spaces/DM/pages/48834013/Campaigns#1.1.2.-LSSTCam-Nightly-Validation-Pipeline
- Check campaign also here  https://rubinobs.atlassian.net/wiki/pages/diffpagesbyversion.action?pageId=48834013&selectedPageVersions=145%2C143
- fov-quicklook : https://usdf-rsp-dev.slac.stanford.edu/fov-quicklook/

## Configuration

In [ ]:
FLAG_DUMP_COLLECTIONS = False
FLAG_DUMP_DATASETS = True
FLAG_DUMP_OBJECTSTABLECOLUMNS = False
FLAG_CUT_OBJECTSMAG = True
FLAG_CUT_OBJECTSSNR = True

In [ ]:
MAGCUT = 28.0
SNRCUT = 5.0

In [ ]:
all_bands = ["u", "g", "r", "i", "z", "y"]
all_bands_colors = ["blue", "green", "red", "orange", "yellow", "purple"]

### Choose instrument

In [ ]:
# instrument = "LSSTCam"

# We focus here on sky fields oberved by LSSTComCam, so we select this camera
instrument = "LSSTComCam"

### Choose options

### For LSSTCam : RubinTV, Campaigns , quicklook
- RubinTV : https://usdf-rsp.slac.stanford.edu/rubintv/summit-usdf/lsstcam
- https://rubinobs.atlassian.net/wiki/spaces/LSSTCOM/pages/467370016/LSSTCam+Commissioning+Planning
- LSSTCam DM campaign : https://rubinobs.atlassian.net/wiki/spaces/DM/pages/48834013/Campaigns#1.1.2.-LSSTCam-Nightly-Validation-Pipeline
- Check campaign also here  https://rubinobs.atlassian.net/wiki/pages/diffpagesbyversion.action?pageId=48834013&selectedPageVersions=145%2C143
- fov-quicklook : https://usdf-rsp-dev.slac.stanford.edu/fov-quicklook/

### For LSSTComCam check here : 
- Check here the collection available : https://rubinobs.atlassian.net/wiki/spaces/DM/pages/226656354/LSSTComCam+Intermittent+Cumulative+DRP+Runs

In [ ]:
if instrument == "LSSTCam":
    repo = "/repo/embargo"
    instrument = "LSSTCam"
    collection_validation = instrument + "/runs/nightlyValidation"
    # collection_quicklook   = instrument + '/runs/quickLookTesting'
    collection_validation = os.path.join(collection_validation, "20250416/d_2025_04_15/DM-50157")
    date_start = 20250415
    date_selection = 20250416
    where_clause = "instrument = '" + f"{instrument}" + "'"
    where_clause_date = where_clause + f"and day_obs >= {date_start}"
    skymapName = "lsst_cells_v1"
    BANDSEL = "i"

elif instrument == "LSSTComCam":
    repo = "/repo/main"
    collection_validation = "LSSTComCam/runs/DRP/DP1/w_2025_10/DM-49359"  # work 2025-05-01
    # collection_validation = "LSSTComCam/runs/DRP/DP1/w_2025_17/DM-50530" #updated 2025-05-02
    date_start = 20241024
    date_selection = 20241211
    skymapName = "lsst_cells_v1"
    where_clause = "instrument = '" + instrument + "'"
    where_clause_date = where_clause + f"and day_obs >= {date_start}"

    NDET = 9
    TRACTSEL = 5063
    BANDSEL = "i"

In [ ]:
collectionStr = collection_validation.replace("/", "_")

## Access to Butler registry

In [ ]:
# Initialize the butler repo:
butler = Butler(repo, collections=collection_validation)
registry = butler.registry

## Create a skymap object and Camera

In [ ]:
skymap = butler.get("skyMap", skymap=skymapName, collections=collection_validation)

In [ ]:
camera = butler.get("camera", collections=collection_validation, instrument=instrument)

## Query for collections in Butler

- remove user collections
- remove calibration products

In [ ]:
# mostly setup for LSSTCam
if FLAG_DUMP_COLLECTIONS:
    for _ in sorted(registry.queryCollections(expression=instrument + "/*")):
        if "/calib/" not in _ and "u/" not in _:
            print(_)

## Query for the dataset types in the Butler

- Refer to the Data Product definition Document to know about the definition of datasets
- https://www.lsst.org/about/dm/data-products
- https://lse-163.lsst.io/
- https://docushare.lsst.org/docushare/dsweb/Get/LSE-163

In [ ]:
if FLAG_DUMP_DATASETS:
    for datasetType in registry.queryDatasetTypes():
        if registry.queryDatasets(datasetType, collections=collection_validation).any(
            execute=False, exact=False
        ):
            # Limit search results to the data products
            if (
                ("_config" not in datasetType.name)
                and ("_log" not in datasetType.name)
                and ("_metadata" not in datasetType.name)
                and ("_resource_usage" not in datasetType.name)
                and ("Plot" not in datasetType.name)
                and ("Metric" not in datasetType.name)
                and ("metric" not in datasetType.name)
            ):
                if "object" in datasetType.name or "Obj" in datasetType.name:
                    print(datasetType)
#                if "source" in datasetType.name or "Source" in datasetType.name:
#                    print(datasetType)

## Region of interest

In [ ]:
lsstcomcam_targets = {}
lsstcomcam_targets["47 Tuc"] = {"field_name": "47 Tuc Globular Cluster", "ra": 6.02, "dec": -72.08}
lsstcomcam_targets["Rubin SV 38 7"] = {"field_name": "Low Ecliptic Latitude Field", "ra": 37.86, "dec": 6.98}
lsstcomcam_targets["Fornax dSph"] = {
    "field_name": "Fornax Dwarf Spheroidal Galaxy",
    "ra": 40.0,
    "dec": -34.45,
}
lsstcomcam_targets["ECDFS"] = {"field_name": "Extended Chandra Deep Field South", "ra": 53.13, "dec": -28.10}
lsstcomcam_targets["EDFS"] = {"field_name": "Euclid Deep Field South", "ra": 59.10, "dec": -48.73}
lsstcomcam_targets["Rubin SV 95 -25"] = {
    "field_name": "Low Galactic Latitude Field",
    "ra": 95.00,
    "dec": -25.0,
}
lsstcomcam_targets["Seagull"] = {"field_name": "Seagull Nebula", "ra": 106.23, "dec": -10.51}

In [ ]:
# the_target = lsstcomcam_targets["Seagull"]
# the_target = lsstcomcam_targets["47 Tuc"] # bad
# the_target = lsstcomcam_targets["Fornax dSph"]
# the_target = lsstcomcam_targets["ECDFS"]


# key = "Seagull"
# key = "Fornax dSph"
key = "ECDFS"
# key = "EDFS"
# key = "47 Tuc"
# key = "Rubin SV 38 7"
# key = "Rubin SV 95 -25"


the_target = lsstcomcam_targets[key]
target_ra = the_target["ra"]
target_dec = the_target["dec"]
target_title = (
    the_target["field_name"] + f" band  {BANDSEL} " + f" (ra,dec) = ({target_ra:.2f},{target_dec:.2f}) "
)
target_point = SpherePoint(target_ra, target_dec, degrees)

## Get list of tracts from the objectTable_tract

In [ ]:
# Find the dimension
print(butler.registry.getDatasetType("objectTable_tract").dimensions)

In [ ]:
datasettype = "objectTable_tract"
therefs = butler.registry.queryDatasets(datasettype, collections=collection_validation)
tractsId_list = np.unique([ref.dataId["tract"] for ref in therefs])
tractsId_list = sorted(tractsId_list)
print(tractsId_list)

## Find the Tract and Patch of the region of interest

- tract in tractNbSel
- patch in patchNbSel

In [ ]:
tract_info = skymap.findTract(target_point)
patch_info = tract_info.findPatch(target_point)
bbox = patch_info.getOuterBBox()

print("Patch bounding box:", bbox)

print("Tract ID :", tract_info.getId())
tractNbSel = tract_info.getId()

print("Patch Index :", patch_info.getIndex(), " , ", patch_info.getSequentialIndex())  # (x, y)
print("Bounding Box", bbox)

patchNbSel = patch_info.getSequentialIndex()

In [ ]:
dataId = {"band": BANDSEL, "tract": tractNbSel, "patch": patchNbSel, "skymap": skymapName}

## The Objects

- Objects are extracted from object detection on deepcoadds
- all bands are included 

In [ ]:
print(butler.registry.getDatasetType("objectTable").dimensions)

In [ ]:
# cannot add a filter on band
where_clause = f"skymap = '{skymapName}' AND tract = {tractNbSel} AND patch = {patchNbSel}"
print(where_clause)

In [ ]:
dataset_refs = list(
    butler.registry.queryDatasets("objectTable", collections=collection_validation, where=where_clause)
)
# Récupère un des refs valides
Nrefs = len(dataset_refs)
print(f"Number of objectTables : {Nrefs}")
ref = dataset_refs[0]
t = butler.get(ref)
Nobj = len(t)
# del t
# gc.collect()
print(f"Total Number of objects {Nobj}")

Oui, dans ce type de objectTable, les colonnes comme g_psfFlux, g_kronFlux, g_cModelFlux, etc., sont des flux calibrés (en nJy ou en unité de calibration interne du pipeline). Pour les convertir en magnitudes AB, tu peux utiliser la formule classique :

In [ ]:
# Utilise ref.datasetType.name pour lister les colonnes disponibles
if FLAG_DUMP_OBJECTSTABLECOLUMNS:
    t_columns = list(butler.get(ref).columns)
    print(t_columns)

In [ ]:
# La constante 31.4 correspond à la conversion AB standard pour un flux exprimé en nanoJanskys (nJy).
# À vérifier selon l’unité exacte utilisée par le pipeline sur ton RSP
# (souvent c’est bien nJy, mais ça peut être autre chose si la calibration a été changée).
# mag = -2.5 * np.log10(flux) + 31.4

## Extract the per-band tables of coordinates

In [ ]:
# Check columns definitions in https://lse-163.lsst.io/
all_radecTable = []

for band in all_bands:
    id_name = "objectId"
    id_parentname = "parentObjectId"
    x_name = f"{band}_centroid_x"
    y_name = f"{band}_centroid_y"
    coord_ra_name = "coord_ra"
    coord_dec_name = "coord_dec"
    ra_name = f"{band}_ra"
    dec_name = f"{band}_dec"
    decl_name = f"{band}_decl"
    raerr_name = f"{band}_raErr"
    decerr_name = f"{band}_decErr"
    extendedness_name = f"{band}_extendedness"
    blendness_name = f"{band}_blendedness"
    psfflux_name = f"{band}_psfFlux"
    psffluxerr_name = f"{band}_psfFluxErr"
    psfmag_name = f"{band}_psfMag"
    psfmagerr_name = f"{band}_psfMagErr"
    psfflux_free_name = f"{band}_free_psfFlux"
    df = t[
        [
            id_name,
            id_parentname,
            x_name,
            y_name,
            coord_ra_name,
            coord_dec_name,
            ra_name,
            dec_name,
            raerr_name,
            decerr_name,
            decl_name,
            extendedness_name,
            blendness_name,
            psfflux_name,
            psffluxerr_name,
        ]
    ]

    # select primary objects
    df_sel = (df[df[id_parentname] == 0]).drop([id_parentname], axis=1)
    # compute magnitude AB

    df_sel[psfmag_name] = df_sel[psfflux_name].apply(nJy_to_ab_mag)
    df_sel[psfmagerr_name] = nJy_err_to_ab_err(df_sel[psfflux_name], df_sel[psffluxerr_name])

    # drop bad magnitudes
    df_sel = df_sel[df_sel[psfmag_name] != 0.0]
    # drop the fluxes
    # df_sel = df_sel.drop([psfflux_name], axis=1)

    # SNR
    snr = df_sel[psfflux_name] / df_sel[psffluxerr_name]

    if FLAG_CUT_OBJECTSSNR:
        df_sel = df_sel[snr > SNRCUT]

    # select bright objects
    if FLAG_CUT_OBJECTSMAG:
        df_sel = df_sel[df_sel[psfmag_name] < MAGCUT]

    df_sel.reset_index(drop=True, inplace=True)

    # save table in the list
    all_radecTable.append(df_sel)

In [ ]:
for idx, band in enumerate(all_bands):
    df_obj = all_radecTable[idx]
    n = len(df_obj)
    print(f"Number of objects in band {band} : {n}")

## Plot Magnitudes and errors

In [ ]:
df_obj

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 4))

for ib, band in enumerate(all_bands):
    color = all_bands_colors[ib]
    df = all_radecTable[ib]
    psfmag_name = f"{band}_psfMag"
    df[psfmag_name].hist(ax=ax, bins=50, histtype="step", color=color, label=band)
ax.legend()
ax.set_yscale("log")
ax.set_xlabel("psfMag")
ax.set_title(f"Magnitude of object in (tract,patch) = ({tractNbSel},{patchNbSel})")
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 6))
for ib, band in enumerate(all_bands):
    color = all_bands_colors[ib]
    df = all_radecTable[ib]
    psfmag_name = f"{band}_psfMag"
    psfmagerr_name = f"{band}_psfMagErr"

    df.plot.scatter(x=psfmag_name, y=psfmagerr_name, ax=ax, marker=".", s=10, c=color, label=band, alpha=0.5)
    # produce a crash
ax.legend()
ax.set_xlabel("psfMag")
ax.set_ylabel("psfMagErr")
ax.set_title(f"Magnitude of Objects in (tract,patch) = ({tractNbSel},{patchNbSel})")
ax.set_aspect("auto")
plt.show()

## Plot cutouts

In [ ]:
import numpy as np
from astropy.coordinates import SkyCoord
import astropy.units as u
from lsst.geom import Point2D, Box2I, Extent2I
from lsst.afw.image import ExposureF


def extract_deepCoadd_cutout(
    ra_deg,
    dec_deg,
    tractInfo,
    patchInfo,
    band,
    butler,
    cutout_size_pixels=200,
    collection=collection_validation,
):
    """
    Extract a square cutout from a deepCoadd image centered on given sky coordinates.

    Parameters
    ----------
    ra_deg : float
        Right ascension in degrees.
    dec_deg : float
        Declination in degrees.
    tractInfo : lsst.skymap.TractInfo
        Tract information object (from SkyMap).
    patchInfo : lsst.skymap.PatchInfo
        Patch information object (from SkyMap).
    band : str
        Photometric band (e.g., 'i', 'r', 'g').
    butler : lsst.daf.butler.Butler
        Butler instance to access LSST data.
    cutout_size_pixels : int, optional
        Size of the cutout in pixels (square region), default is 200.
    collection : str, optional
        Name of the collection containing deepCoadd data.

    Returns
    -------
    cutout : lsst.afw.image.ExposureF
        The extracted image cutout.
    wcs : lsst.afw.geom.SkyWcs
        World Coordinate System associated with the cutout.
    metadata : lsst.daf.base.PropertyList
        FITS metadata header for the cutout image.
    """

    # Build dataId for the deepCoadd image
    tract = tractInfo.getId()
    patch = patchInfo.getIndex()
    dataId = dict(tract=tract, patch=patch, band=band)

    # Retrieve the deepCoadd exposure
    exposure = butler.get("deepCoadd", dataId=dataId, collections=collection)

    # Convert sky coordinates (RA, Dec) to pixel coordinates using WCS
    coord = SkyCoord(ra=ra_deg * u.deg, dec=dec_deg * u.deg)
    wcs = exposure.getWcs()
    pixel_center = wcs.skyToPixel(coord)

    # Define a square bounding box around the target pixel
    half_size = cutout_size_pixels // 2
    bbox = Box2I(
        Point2D(int(pixel_center.getX()) - half_size, int(pixel_center.getY()) - half_size),
        Extent2I(cutout_size_pixels, cutout_size_pixels),
    )

    # Extract the cutout from the original exposure
    cutout = exposure.Factory(exposure, bbox, deep=True)

    return cutout, cutout.getWcs(), cutout.getMetadata()